# Outliers and Missing Data

## 1. 실험 환경 변수

### 사용 패키지

In [ ]:
from datetime import datetime
from typing import Dict, List, Tuple

import geopandas as gpd
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
from matplotlib.ticker import FuncFormatter

from metr.components import TrafficData, Metadata
import geopandas as gpd

plt.rcParams["font.family"] = "AppleGothic"
plt.rcParams["axes.unicode_minus"] = False  # 음수 부호 깨짐 방지

### 주요 데이터 경로

In [ ]:
MAP_DATA_OF_SENSORS = "../datasets/metr-imc/nodelink/imc_link.shp"
TRAFFIC_RAW_PATH = "../datasets/metr-imc/metr-imc.h5"
METADATA_RAW_PATH = "../datasets/metr-imc/metadata.h5"
OUTLIER_OUTPUT_DIR = "./output/outlier_processed"
INTERPOLATED_OUTPUT_DIR = "./output/interpolated"

## 2. Raw Data

### Raw 데이터 로딩

Raw 데이터는 원래 인천시에서 제공되었던 형태에서 현재 METR-LA와 같은 형태로 변환되어 있다. 행은 시간을 나타내고 열은 각 도로의 센서를 나타낸다. 형식은 전국표준노드링크의 Link ID를 나타내며 string 형식으로 지정되어 있다.

본 연구에서는 2024년 3월부터 2024년 9월까지의 데이터를 사용한다. 데이터가 어느정도 깔끔해서의 이유도 있지만 공식적으로는 연구 시작 지점에서 RNN 모델의 성능이 가장 높게 나오는 데이터의 길이이기 때문이다.

In [ ]:
raw_df = TrafficData.import_from_hdf(TRAFFIC_RAW_PATH).data
raw_df = raw_df.loc["2024-03-01":"2024-10-01", :]
raw_df

원래 표준노드링크는 전국의 데이터를 포함하며 아래는 그 중 인천시에 해당하는 데이터만 추출한 데이터이다. 해당 데이터는 2024년 3월 25일에 최신화된 데이터이다.

In [ ]:
geo_gdf = gpd.read_file(MAP_DATA_OF_SENSORS)
print(geo_gdf.shape)
geo_gdf.explore()

하지만, 인천시에서는 모든 도로에 대해 교통량을 측정하지 않으며 교통량을 측정하고 있는 도로는 다음과 같다

In [ ]:
# geo_gdf에서 LINK_ID가 raw_df.columns에 있는 것만 남기기
geo_gdf_with_traffic: gpd.GeoDataFrame = geo_gdf[geo_gdf["LINK_ID"].isin(raw_df.columns)]

print(geo_gdf_with_traffic.shape)
geo_gdf_with_traffic.explore()

메타데이터는 표준노드링크에 명시된 내용을 추출했으며 각 센서에 해당하는 도로의 정보를 포함한다. 본 연구에서는 도로 허용 용량을 계산할 때에만 사용한다.

In [ ]:
raw_metadata_df = Metadata.import_from_hdf(METADATA_RAW_PATH).data
raw_metadata_df

### Description of Raw Data

In [ ]:
print("Shape of the dataset:", raw_df.shape)
print("Description of the Incheon dataset")
print(raw_df.reset_index(drop=True).melt()["value"].describe().apply(lambda x: format(x, '.4f')))
print()

total_missing_values_count = raw_df.isnull().sum().sum()
print("Total missing values count:", total_missing_values_count, f"({total_missing_values_count / raw_df.size * 100:.4f}%)")
print()

sensors_with_mv = raw_df.isnull().any(axis=0)
print("Sensors with missing values count:", sensors_with_mv.sum(), f"({sensors_with_mv.sum() / raw_df.shape[1] * 100:.4f}%)")
print("Sensors with no missing values count:", (~sensors_with_mv).sum(), f"({(~sensors_with_mv).sum() / raw_df.shape[1] * 100:.4f}%)")

### Plots of Raw Data

In [ ]:
def plot_mean_by_time(df: pd.DataFrame, title: str, size: Tuple=(10, 6)):
    groups = df.groupby([df.index.hour, df.index.minute, df.index.second])
    mean_df = groups.apply(lambda x: x.stack().mean())
    mean_df.index = [
        datetime(year=1970, month=1, day=1, hour=h, minute=m, second=s)
        for h, m, s in mean_df.index
    ]
    plot_by_time(mean_df, title, size)


def plot_by_time(df: pd.DataFrame, title: str, size: Tuple=(12, 6)):
    plt.figure(figsize=size)
    sns.lineplot(x=df.index, y=df.values)
    # plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%H:%M:%S"))

    plt.xlabel("Time")
    plt.ylabel("Average Value")
    plt.title(title)

    plt.grid(True)

    plt.show()


def plot_by_days(df: pd.DataFrame, name: str):
    data_list = []

    for day in range(7):
        day_data = df[df.index.dayofweek == day]
        numeric_cols = day_data.select_dtypes(include=[np.number]).columns
        day_numeric = day_data[numeric_cols]
        values = day_numeric.values.flatten()
        values = values[~np.isnan(values)]
        df_day = pd.DataFrame({"Volumes": values, "Days": day})
        data_list.append(df_day)

    df_all = pd.concat(data_list, ignore_index=True)

    # 요일 레이블 매핑
    day_labels = {0: "Mon", 1: "Tue", 2: "Wed", 3: "Thu", 4: "Fri", 5: "Sat", 6: "Sun"}
    df_all["Days"] = df_all["Days"].map(day_labels)
    df_all["Days"] = pd.Categorical(
        df_all["Days"],
        categories=["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"],
        ordered=True,
    )

    plt.figure(figsize=(10, 6))
    sns.boxplot(
        x="Days",
        y="Volumes",
        data=df_all,
        order=["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"],
    )
    plt.xlabel("Days")
    plt.ylabel("Traffic Volumes")
    plt.title(f"Data by Day in {name}")
    plt.grid(True)

    ax = plt.gca()
    formatter = FuncFormatter(lambda x, pos: f"{int(x):,}")
    ax.yaxis.set_major_formatter(formatter)

    plt.show()


def plot_hist(
    df: pd.DataFrame,
    name: str,
    exclude_zero: bool = False,
    y_max: float = None,
    x_max: float = None,
    bins: int = 50,
):
    values = df.values.flatten()
    values = values[~np.isnan(values)]

    if exclude_zero:
        values = values[values != 0]

    if x_max is not None:
        bin_edges = np.histogram_bin_edges(
            values, bins=bins, range=(values.min(), x_max)
        )
        bin_edges = np.append(bin_edges, np.inf)
    else:
        bin_edges = bins

    plt.figure(figsize=(10, 6))
    sns.histplot(values, bins=bin_edges, kde=True)
    plt.xlabel("Value")
    plt.title(f"Histogram of {name}")
    plt.grid(True)

    if y_max is not None:
        plt.ylim(top=y_max)

    ax = plt.gca()
    formatter = FuncFormatter(lambda x, _: f"{int(x):,}")
    ax.yaxis.set_major_formatter(formatter)
    ax.xaxis.set_major_formatter(formatter)

    plt.show()

아래 그래프는 전체 데이터를 평균을 내고 추세를 본 것이다. 일반적으로 대부분의 각 센서의 데이터는 아래와 같은 형태를 보인다.

In [ ]:
plot_mean_by_time(raw_df, "Plots by Time")

아래는 요일 별 데이터의 분포를 나타낸 것이다. 2024년 3월 이전의 데이터는 말도 안 되는 값(>100000)들이 많이 있는 것을 확인하기도 했지만 현재 범위(2024-03 ~ 2024-09)의 데이터는 극단적인 값들은 크게 없는 것으로 보인다.

In [ ]:
plot_by_days(raw_df, "Plots by Days")

아래는 데이터 구간 별 데이터 분포를 나타낸다. 20000만개 이상은 그래프에 표시하지 않았지만 대부분의 데이터는 교통량이 2000아래에 분포해 있다. 또한 교통량이 2000이상 넘어가는 도로는 대체로 고속도로이다.

In [ ]:
plot_hist(raw_df, "Data Histogram", y_max=20000)

일반적으로 데이터의 그래프는 아래와 같다. 특정 기간에서 데이터가 증가한 것을 확인할 수 있다. 그 외에도 어떤 그래프의 경우 말도 안되는 값을 가지는 경우도 확인할 수 있다.

In [ ]:
target_data = raw_df.iloc[:, 1]
plot_by_time(target_data, f"Plots by Time: {target_data.name}")
geo_gdf[geo_gdf["LINK_ID"] == target_data.name].explore(
    style_kwds={
        "color": "red",  # 선 색상
        "weight": 5,  # 선 굵기 (픽셀 단위)
    },
)

In [ ]:
# 고속도로
target_data = raw_df.loc[:, "1610002900"]
plot_by_time(target_data, f"Plots by Time: {target_data.name}")
geo_gdf[geo_gdf["LINK_ID"] == target_data.name].explore(
    style_kwds={
        "color": "red",  # 선 색상
        "weight": 5,  # 선 굵기 (픽셀 단위)
    },
)

2024년 3월 이전의 데이터는 100,000이 넘어가는 비정상적 데이터도 확인했지만 현재 범위의 데이터에서는 비정상적으로는 보이지 않는다

In [ ]:
idx = -1

In [ ]:
while idx < raw_df.shape[1]:    
    idx += 1
    target_data = raw_df.iloc[:, idx]
    if target_data[target_data > 10000].any():
        break

plot_by_time(target_data, f"Plots by Time: {target_data.name}")
geo_gdf[geo_gdf["LINK_ID"] == target_data.name].explore(
    style_kwds={
        "color": "red",  # 선 색상
        "weight": 5,  # 선 굵기 (픽셀 단위)
    },
)
        

"1640332801"과 같은 센서의 경우 결측치가 대부분이고 유효한 값은 거의 없는 것으로 보인다. 해당 위치의 경우 차량통행이 꽤 있는 곳이며 측정이 제대로 이루어지지 않았음을 

In [ ]:
idx = -1

In [ ]:
while idx < raw_df.shape[1]:    
    idx += 1
    target_data = raw_df.iloc[:, idx]
    if target_data.isna().sum() > 3000:
        break

plot_by_time(target_data, f"Plots by Time: {target_data.name}")
geo_gdf[geo_gdf["LINK_ID"] == target_data.name].explore(
    style_kwds={
        "color": "red",  # 선 색상
        "weight": 5,  # 선 굵기 (픽셀 단위)
    },
)

## 3. Outliers

### 기본 Outlier 처리

앞서 본 연구에서 다음 이상치 처리를 기본으로 적용한다.

1. 결측치 주변 0을 제거
  경험적 데이터 분석 결과를 바탕으로 결측치 주변 0이 
2. 이론적 도로 허용 용량 기반 제거 (1.5배 또는 2.0배까지 허용)

In [ ]:
from songdo_rnn.preprocessing.outlier import remove_base_outliers

In [ ]:
result_path = remove_base_outliers(
    start_datetime="2024-03-01 00:00:00",
    end_datetime="2024-09-30 23:00:00",
    traffic_capacity_adjustment_rate=2.0,
    raw_path=TRAFFIC_RAW_PATH,
    metadata_path=METADATA_RAW_PATH,
    output_dir=OUTLIER_OUTPUT_DIR,
)

In [ ]:
base_df = TrafficData.import_from_hdf(result_path).data
base_df

### 기본 Outlier 처리 결과 시각화

In [ ]:
def compare_fromto_df(df1: pd.Series, df2: pd.Series, title: str = ""):
    plt.figure(figsize=(12, 6))
    df1.plot(label="Original Data", color="red")
    df2.plot(label="Processed Data", color="blue")
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
idx = -1

In [ ]:
idx += 1

original_data = raw_df.iloc[:, idx]
target_data = base_df.iloc[:, idx]

compare_fromto_df(original_data, target_data, title=f"Origin vs Target: {original_data.name}")

In [ ]:
idx = -1

In [ ]:
while idx < raw_df.shape[1]:
    idx += 1
    original_data = raw_df.iloc[:, idx]
    target_data = base_df.iloc[:, idx]

    if original_data.isna().sum() == target_data.isna().sum():
        continue

    print(original_data.name, ":" , original_data.isna().sum())
    print(target_data.name, ":", target_data.isna().sum())

    compare_fromto_df(original_data, target_data, title=f"Origin vs Target: {original_data.name}")

    break

극단적으로 결측치가 많은 경우

In [ ]:
target_sensor_idx = "1640332801"
original_data = raw_df[target_sensor_idx]
target_data = base_df[target_sensor_idx]

compare_fromto_df(original_data, target_data, title=f"Origin vs Target: {original_data.name}")

In [ ]:
odata = original_data[original_data.notna() & original_data > 0]
tdata = target_data[odata.index]
print(odata.name)
print(tdata.name)
for o, t in zip(odata.items(), tdata.items()):
    print(o[0], o[1], t[1])

아래 코드는 0제거 외에 허용 용량에 따른 제거된 사례를 확인하는 코드임. 테스트 상 2.0배를 넘어가는 경우는 종종 있지만 2.5를 넘어가지는 않음. 대체로 속도가 바뀌는 지점의 도로에서 과속이 많이 발생하고 이에 따라 허용 용량 이상으로 차량이 통과하는 것으로 보임. 따라서, 실제 통과 차량의 한계를 얼마나 정할 것인지 정할 필요가 있음.

In [ ]:
idx = -1

In [ ]:
while idx < raw_df.shape[1] - 1:
    idx += 1
    original_data = raw_df.iloc[:, idx]
    target_data = base_df.iloc[:, idx]

    if original_data.isna().sum() == target_data.isna().sum():
        continue

    not_0_and_na = original_data[original_data != 0]
    not_0_and_na = not_0_and_na[not_0_and_na.notna()]
    na_target = target_data[not_0_and_na.index]
    if na_target.isna().sum() == 0:
        continue

    print(original_data.name, "(Origin):" , original_data.isna().sum())
    print(target_data.name, "(Target):", target_data.isna().sum())
    metadata = raw_metadata_df[raw_metadata_df["LINK_ID"] == original_data.name]
    print(metadata)

    speed_limit = metadata["MAX_SPD"].values[0]
    alpha = 10 * (100 - speed_limit)
    if speed_limit > 100:
        alpha /= 2
    lane_count = metadata["LANES"].values[0]
    adjustment_rate = 2.5
    original_capacity = (2200 - alpha) * lane_count
    capacity = original_capacity * adjustment_rate
    print("Capacity:", capacity, f"({original_capacity} x {adjustment_rate})")
    exceed_capacity_data = target_data[target_data > capacity]
    print(exceed_capacity_data if exceed_capacity_data.size > 0 else "No exceed capacity data")

    compare_fromto_df(original_data, target_data, title=f"Origin vs Target: {original_data.name}")
    
    break

print(original_data.name)
geo_gdf[geo_gdf["LINK_ID"] == original_data.name].explore()

### 여러 Outlier 처리

In [ ]:
from metr.components.metr_imc.outlier import (
    OutlierProcessor,
    HourlyInSensorZscoreOutlierProcessor,
    InSensorZscoreOutlierProcessor,
    MADOutlierProcessor,
    TrimmedMeanOutlierProcessor,
    WinsorizedOutlierProcessor,
)
from songdo_rnn.preprocessing.outlier import (
    get_outlier_removed_data_list,
    remove_outliers,
)

In [ ]:
outlier_processors: List[OutlierProcessor] = [
    HourlyInSensorZscoreOutlierProcessor(),
    InSensorZscoreOutlierProcessor(),
    WinsorizedOutlierProcessor(),
    TrimmedMeanOutlierProcessor(),
    MADOutlierProcessor(),
]

outlier_processors[0].name = "hzscore"
outlier_processors[1].name = "zscore"
outlier_processors[2].name = "winsor"
outlier_processors[3].name = "trimm"
outlier_processors[4].name = "mad"

In [ ]:
result_paths = remove_outliers(
    data=base_df,
    outlier_processors=outlier_processors,
    output_dir=OUTLIER_OUTPUT_DIR,
)
result_paths

In [ ]:
outlier_results = get_outlier_removed_data_list()
len(outlier_results)